# 2/1/17

`FEProblemBase::reinitMaterials` only calls property computation for Material objects currently active on the given subdomain so that's good. However, it's possible that material objects "active" on the subdomain aren't actually being used in any computing objects like kernels, etc. So we would like to do some additional checking.

Alright, let's say we're computing the residual thread. Then assuming we cannot compute properties in a material in isolation, we would like to do the next best thing: only call `computeQpProperties` for materials that have actually been asked to supply properties to kernels, dg_kernels, boundary_conditions, and interface_kernels. 

So what am I doing as of commit d7dbfe5? I am determining the `needed_mat_props` through `ComputeResidualThread::subdomainChanged()` -> `FEProblemBase::prepareMaterials`. In the latter method, we first ask all materials--if there are _any_ materials active on the block--to  update their material property dependencies and then we ask materials on the boundaries of the subdomain to also update their dependencies. Note that this could lead to a boundary material object getting asked to update their material property dependencies twice because we first pass to `MaterialWarehouse::updateMatPropDependenceyHelper` all material objects as long as there is any one material object active in a block sense on the subdomain, and then we pass active material boundary objects. But this overlap doesn't matter so much because our `needed_mat_props` is a set, so if we try to insert the same material properties multiple times, it will silently and correctly fail. Note,  however, that this could also pass `needed_mat_props` from material objects not on the current block, so that needs to be changed.

So what happens in `MaterialWarehouse::updateMatPropDependencyHelper`? We add `mp_deps` from `MaterialPropertyInterface::getMatPropDependencies`. However, it should be noted that this is only done for  <i>material objects</i>. Is this fine? Well let's figure it out. It returns `_material_property_dependencies` which is a set added to by calling `addMatPropDependency`. Now this gets called when the object that inherits from `MaterialPropertyInterface` calls its own `getMaterialProperty` method. So I hypothesize that in my simple test, if I ask to have a material property in my kernel object with `getMaterialProperty` that will not register in any material objects list of `_material_property_dependencies` and consequently `computeQpProperties` will never get called. I will test that the next time I sit down at my comp.

# 2/2/17

Three tests:

- Run with _one_ material that doesn't supply any properties. Desired behavior: `computeQpProperties` does not get called. Expected to **Pass**. With devel MOOSE: expected to **Fail** (expected **change**)
- Run _two_ materials, one that supplies properties, another that does not. Desired behavior: `computeQpProperties` does not get called for the material not supplying properties while the other one does. Expected behavior:  both materials compute methods get called. **Fail**. With devel MOOSE: expected to **Fail** (expected to **not change**)
- Run with a kernel that uses a material property and an elemental aux variable that does not. Desired behavior: `computeQpProperties` should get called through the residual and jacobian threads but not through the aux kernel thread. Expected  to **Pass**. With devel MOOSEE: expected to **Fail** (expected **change**)

Calls to computeProperties:

1. ComputeResidualThread
2. ComputeResidualThread
0th nonlinear residual printed
3. ComputeJacobianThread
4. ComputeResidualThread
0th linear residual printed
5. ComputeResidualThread
1st linear residual printed
6. ComputeResidualThread
7. ComputeResidualThread
1st nonlinear residual printed
8. ComputeElemAuxVarsThread -> Actually this is fine because this is the Aux Kernel that is created for outputting the material property

Number of calls: 8
1. 1-4
2. 5-8
...
7. 25-28
8. 29-32

Failed tests:

- random.material_serial
- controls\*

Failed but now passing:

- element\_aux\_boundary
- bnd\_material\_test
- elem\_aux\_bc\_on\_bound
- output.boundary
- multiplicity
- material_point_source_test
- line\_material\_sampler

# 2/6/17

Ok my new test is failing with threads and I don't really know why. It seems like the number of calls to computing threads should be the same...

Calls to computeProperties:

1. ComputeResidualThread
2. ComputeResidualThread
0th nonlinear residual printed
3. ComputeJacobianThread
4. ComputeResidualThread
0th linear residual printed
5. ComputResidualThread
1st linear residual printed
6. ComputeResidualThread
7. ComputeResidualThread
1st nonlinear residual printed
8. ComputeElemAuxVarsThread

Yep so thread computing pattern is the exact same. How about whether the material is the same location in memory every time?

- 0x7fed90 (1, 2, 8)
Increments:
1-4, 5-8, 9-12 -> average of 10.5 which is what is observed in the output
- 0x810b10 (3, 4, 5, 6, 7